### Домашнее задание «Функции и работа с данными»

#### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [7]:
import pandas as pd

def rating_gradation(param):
    """оценка 2 и меньше - низкий рейтинг
       оценка 4 и меньше - средний рейтинг
       оценка 4.5 и 5 - высокий рейтинг"""
    result = 'не определен'
    if param <= 2 :
        result = 'низкий рейтинг'
    elif param <= 4 :
        result = 'средний рейтинг'
    elif param > 4 :
        result = 'высокий рейтинг'    
    return result

ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()
ratings['class'] = ratings['rating'].apply(rating_gradation)
ratings


,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг
...,...,...,...,...,...
99999,671,6268,2.5,1065579370,средний рейтинг
100000,671,6269,4.0,1065149201,средний рейтинг
100001,671,6365,4.0,1070940363,средний рейтинг
100002,671,6385,2.5,1070979663,средний рейтинг


#### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [66]:
import re

geo_data = {'Центр': ['москва', 'тула', 'ярославль', 'старый оскол', 'воронеж'],
'Северо-Запад': ['петербург', 'псков', 'мурманск', 'вологда'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'],
'Урал': ['екатеринбург', 'челябинск', 'тюмень', 'магнитогорск', 'сургут'],            
}

geo_data_patern = {}

for name, value in geo_data.items():
    pattern = ''
    for item in value:
        if pattern == '':
            pattern = r'('
        else : pattern +=  '|'        
        pattern +=item[0:-1]+'[а-я]+'
    pattern +=  ')'  
    geo_data_patern[name] = pattern 

#print(geo_data_patern)

def geo_class(keyword):
    """гео-классификатор, который каждой строке сможет выставить 
    географическую принадлежность определенному региону. 
    Т. е. если поисковый запрос содержит название города региона, 
    то в столбце ‘region’ пишется название этого региона. 
    Если поисковый запрос не содержит названия города, то ставим ‘undefined’."""
    
    result = 'undefined'
    for name, value in geo_data_patern.items():
        #print(value, keyword)
        if re.findall(value, keyword):
            result =  name
    return result    

data = pd.read_csv('ml-latest-small/keywords.csv')
data.head()

data['region'] = data['keyword'].apply(geo_class)
data[data['region'] != 'undefined']

,keyword,shows,region
127,авито москва,979292,Центр
143,эхо москвы,889657,Центр
197,погода в москве,745745,Центр
370,авито ру санкт петербург,425134,Северо-Запад
414,погода в москве на 14 дней,400914,Центр
...,...,...,...
99849,малахитовая шкатулка,3650,Центр
99890,яндекс метро москва,4548,Центр
99922,авто ру ярославль,3712,Центр
99944,гибдд тюмени официальный сайт,3612,Урал


#### Задание 3
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [144]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

def production_year(title):
    """извлекаем год"""
    result =  re.findall(r'[1-2][0-9]{3}', title)
    
    if len(result) == 1:
        if int(result[0]) > 1950 :return result[0]
        else:  return 1900
    elif len(result) == 2 :
        if int(result[1]) > 1950:return result[1]
        else:  return 1900
    else: return None
    
movies['years'] = movies['title'].apply(production_year)
movies

joined = ratings.merge(movies, on='movieId', how='left')

joined.groupby('years').rating.mean().sort_values(ascending=False)

years
1957    4.014241
1972    4.011136
1952    4.000000
1954    3.994220
1951    3.983539
          ...   
2011    3.401188
2015    3.371028
2013    3.356973
2000    3.355945
2016    3.217742
Name: rating, Length: 67, dtype: float64